In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import geopandas as gpd
import h3pandas 
sys.path.append('../../../reproducible_figures/scripts/')
import rep_fig_vis as rfv
sys.path.append('../../scripts/')
import create_dataset_utils as cdu
sys.path.append('../../../cnn-land-cover/scripts/')
import land_cover_analysis as lca 
import land_cover_visualisation as lcv

sys.path.append('../../scripts/')
import create_dataset_utils as cdu
import paired_embeddings_models as pem
import data_vis_utils as dvu
import shapely
from tqdm import tqdm, tqdm_notebook
import os
import pandas as pd

Font size is set to 10


In [15]:
tmp = pem.load_stats(timestamp='2024-03-02-0232')

Loaded stats from PECL-ImEn_2024-03-02-0232_vnum-243_stats.pkl at /Users/t.vanderplas/models/PECL/stats/


In [51]:
tmp_df, tmp_details = dvu.create_df_list_timestamps(list_ts=[
    '2024-03-01-0055', 
    '2024-03-01-0109', 
    '2024-03-01-0125', 
    '2024-03-01-0146', 
    '2024-03-01-0205', 
    '2024-03-01-0222',
    ])

Loaded stats from PECL-ImEn_2024-03-01-0055_vnum-205_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0109_vnum-206_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0125_vnum-207_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0146_vnum-208_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0205_vnum-209_stats.pkl at /Users/t.vanderplas/models/PECL/stats/
Loaded stats from PECL-ImEn_2024-03-01-0222_vnum-210_stats.pkl at /Users/t.vanderplas/models/PECL/stats/


In [53]:
tmp_df

,timestamp,n_species,n_enc_channels,n_bands,n_layers_mlp_resnet,n_layers_mlp_pred,pretrained_resnet,freeze_resnet,optimizer_name,resnet_version,...,time_created,name_train_loss,n_epochs_converged,val_top_20_acc,val_top_10_acc,val_top_5_acc,val_top_1_acc,val_bce_loss,val_mse_loss,val_pecl-softmax_loss
0,2024-03-01-0055,62,256,4,1,2,seco,True,Adam,18,...,2024-03-01-0035:19,pred-bce_pecl-softmax,59,0.854664,0.707836,0.608955,0.466418,0.227386,0.012668,0.000000
1,2024-03-01-0109,62,256,4,1,2,seco,True,Adam,18,...,2024-03-01-0055:27,pred-bce_pecl-softmax,35,0.851866,0.711567,0.618657,0.470149,0.228554,0.012533,0.391132
2,2024-03-01-0125,62,256,4,1,2,seco,True,Adam,18,...,2024-03-01-0109:01,pred-bce_pecl-softmax,46,0.859328,0.704104,0.629104,0.391791,0.227243,0.012176,0.000000
3,2024-03-01-0146,62,256,4,1,2,seco,True,Adam,18,...,2024-03-01-0125:40,pred-bce_pecl-softmax,61,0.856530,0.704851,0.632090,0.414179,0.226128,0.012054,0.392806
4,2024-03-01-0205,62,256,4,1,2,seco,True,Adam,18,...,2024-03-01-0146:28,pred-bce_pecl-softmax,55,0.861194,0.708582,0.634328,0.429104,0.223049,0.011567,0.000000
5,2024-03-01-0222,62,256,4,1,2,seco,True,Adam,18,...,2024-03-01-0205:37,pred-bce_pecl-softmax,46,0.860821,0.710821,0.638060,0.458955,0.222413,0.011369,0.389009


In [116]:
df_print = dvu.create_printable_table(df=tmp_df, hparams_use=tmp_details[1], metrics_use=tmp_details[2],
                           metric_optimise=tmp_details[3], hparam_show='alpha_ratio_loss')
df_print

Font size is set to 10
Hyperparameter alpha_ratio_loss has 2 unique values
Hyperparameter seed_used has 3 unique values
{'n_species': 62, 'n_enc_channels': 256, 'n_bands': 4, 'n_layers_mlp_resnet': 1, 'n_layers_mlp_pred': 2, 'pretrained_resnet': 'seco', 'freeze_resnet': True, 'optimizer_name': 'Adam', 'resnet_version': 18, 'pecl_distance_metric': 'softmax', 'pred_train_loss': 'bce', 'lr': 0.001, 'pecl_knn': 2, 'pecl_knn_hard_labels': True, 'training_method': 'pred_and_pecl', 'normalise_embedding': 'l2', 'use_mps': True, 'use_lr_scheduler': True, 'batch_size_used': 64, 'verbose': 1, 'name_train_loss': 'pred-bce_pecl-softmax'}
('Top 10 acc', 'mean')
('Top 10 acc', 'sem')
('Top 5 acc', 'mean')
('Top 5 acc', 'sem')
('MSE loss', 'mean')
('MSE loss', 'sem')


(                 Top 10 acc        Top 5 acc        MSE loss     
                        mean    sem      mean    sem     mean  sem
 alpha_ratio_loss                                                 
 0.0                   0.707  0.001     0.624  0.008    0.012  0.0
 0.1                   0.709  0.002     0.630  0.006    0.012  0.0,
    alpha_ratio_loss         Top 10 acc          Top 5 acc           MSE loss
 0               0.0  0.707 $\pm$ 0.001  0.624 $\pm$ 0.008  0.012 $\pm$ 0.000
 1               0.1  0.709 $\pm$ 0.002  0.630 $\pm$ 0.006  0.012 $\pm$ 0.000)

In [117]:
df_print[0]

Top 10 acc        Top 5 acc        MSE loss     
                       mean    sem      mean    sem     mean  sem
alpha_ratio_loss                                                 
0.0                   0.707  0.001     0.624  0.008    0.012  0.0
0.1                   0.709  0.002     0.630  0.006    0.012  0.0

In [118]:
df_print[1]

,alpha_ratio_loss,Top 10 acc,Top 5 acc,MSE loss
0,0.0,0.707 $\pm$ 0.001,0.624 $\pm$ 0.008,0.012 $\pm$ 0.000
1,0.1,0.709 $\pm$ 0.002,0.630 $\pm$ 0.006,0.012 $\pm$ 0.000
